<a href="https://colab.research.google.com/github/esraaelelimy/carbon_footprint/blob/main/q_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Implementation

In [140]:
!pip install gymnasium

In [141]:
# ACTION Reduce by 10%, 20% etc -> increase action space
import argparse
import os
import random
import time
from distutils.util import strtobool

import gymnasium as gym
from gym.spaces import Discrete, Box
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# from stable_baselines3.common.buffers import ReplayBuffer
from torch.utils.tensorboard import SummaryWriter

possible_actions = ["increase by level 1","increase by level 2","increase by level 3","maintain",
           "decrease by level 1","decrease by level 2","decrease by level 3"]

action_for_each_state = ()

In [142]:
# DOUBTS: Should co2 be considered a state space
# state_space1 = (gamma, pue, tdp_watts, config, chips) 
# state_space2 = (gamma1, pue1, tdp_watts1, config1, chips1) 
# difference = ce.state1 - ce.state2 
# difference is positive -> + reward
# difference is negative -> - reward  



# goal -> 395-405 -> terminal state reached 
# budget limit = 100
# current e = 500

# diff = 400



# Environment Implementation 
class env():
    # start from state and then take an action to return next state and the reward in the next state
    def __init__(self, curr_state,termination_co2):
        # 7 actions can be taken 
        self.action_space = Discrete(7)          
        self.curr_state = curr_state
        self.termination_co2 = termination_co2
        # self.info_action = (0,0,0,0,0)
        super().__init__()

    
    def carbon_emissions(self,curr_state):
        product = 1
        # state_space1 = (gamma, pue, tdp_watts, config, chips) 
        for i in range(len(curr_state)):
            product *= curr_state[i]
        return product
    
       
    def step(self,info_action):
        # state_space1 = (gamma, pue, tdp_watts, config, chips) 
        # info_action = (0,4,5,6,3)
        reward = 0
        prev_co2 = self.carbon_emissions(self.curr_state)
        
        # each state component takes an action 
        state_list = list(self.curr_state)
        actions_list = list(info_action)
        
        for i in range(len(state_list)):
            state_list[i] += actions_list[i] 
            
        self.curr_state = state_list
        
        new_state = self.curr_state
        
        new_co2 = self.carbon_emissions(new_state)
        
        old_diff = abs(self.termination_co2 - prev_co2)
        new_diff = abs(self.termination_co2 - new_co2)
        
        if new_diff <= 0.1*self.termination_co2:
            done = True
            reward += 5
        else:
            # 100 - 500 = 400 -> old diff
            # 100 - 200 = 200 -> new diff 
            # 200 - 400  = - 200 
            if new_diff - old_diff >0: 
                reward -= 1
            else:
                reward += 1
            done = False
        # info could be actions_list ?????????
        info = {}
        return self.curr_state, reward, done, info
        
    # difference between reset and init
    def reset(self, curr_state,termination_co2):
        self.action_space = Discrete(7)   
        self.curr_state = curr_state
        self.termination_co2 = termination_co2
#         self.info_action = (0,0,0,0,0)
        return curr_state


In [143]:
state = [10,20,30,40,50]
env1 = env(state,100)
prod = env1.step([0,1,2,3,4])
print(prod)

([10, 21, 32, 43, 54], -1, False, {})


#   RL Model implementation using Pytorch

In [144]:
class QNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 35),
        )

    def forward(self, x):
        return self.network(x)

In [145]:
q_network = QNetwork()
optimizer = optim.Adam(q_network.parameters(), lr=1e-3)
target_network = QNetwork()

In [146]:
# 40 * 8 * 221 = 320*221 = 70,720 * 20 * 100 = 141,440,000
state = torch.tensor(state,dtype= torch.float32)
qval = q_network(state)


# Agent Implementation

In [147]:
# python dqn.py --total-timesteps 500000 \ 220
#     --learning-rate 2.5e-4 \
#     --buffer-size 10000 \
#     --gamma 0.99 \ 0.9
#     --target-network-frequency 500 \ 30
#     --max-grad-norm 0.5 \
#     --batch-size 128 \
#     --start-e 1 \
#     --end-e 0.05 \
#     --exploration-fraction 0.5 \
#     --learning-starts 10000 \ 15
#     --train-frequency 1 
#     --tau / 0.9


In [148]:
def concise(states):
    # tensor of 35 length
    tensor_35 = torch.tensor(states)
    # Reshape the tensor into a 2D array of size 7x5
    tensor_2d = tensor_35.reshape(5, 7)
    # Convert the tensor into a NumPy array
    array_2d = tensor_2d.numpy()
    max_indices = np.argmax(array_2d, axis=1)

    return max_indices

In [149]:
def concise_target(states):
    # tensor of 35 length
    tensor_35 = torch.tensor(states)
    # Reshape the tensor into a 2D array of size 7x5
    tensor_2d = tensor_35.reshape(5, 7)
    # Convert the tensor into a NumPy array
    array_2d = tensor_2d.numpy()
    max_arr = []
    for i in range(len(array_2d)):
        max_val =np.amax(array_2d[i])
        max_arr.append(max_val)
    return max_arr

In [150]:
concise(qval)

/var/folders/0g/5jl4b1d12lvc6g70prn2_rnr0000gn/T/ipykernel_6158/1293452933.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor_35 = torch.tensor(states)


array([0, 1, 5, 3, 2])

In [151]:
# 5 rows and 7 columns
def rand_action():
    rand_indices = []
    for i in range(5):
        random_num = random.randint(0,6)
        rand_indices.append(random_num)
    return rand_indices

In [152]:
concise(qval)
print(rand_action())

[5, 6, 5, 5, 3]


/var/folders/0g/5jl4b1d12lvc6g70prn2_rnr0000gn/T/ipykernel_6158/1293452933.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor_35 = torch.tensor(states)


In [239]:
def i_to_val(tensor_array, actions):
    # [0,1,2,3,4]
    # tensor_array 35
    # actions -> index
    tensor_35 = torch.tensor(tensor_array)
    # Reshape the tensor into a 2D array of size 7x5
    tensor_2d = tensor_35.reshape(5, 7)
    # Convert the tensor into a NumPy array
    array_2d = tensor_2d.numpy()
    print(array_2d)
    old_val = []
    for i in range(len(actions)):
        old_val.append(array_2d[i][actions[i]])
    return torch.tensor(old_val,requires_grad = True)
    

In [240]:
i_to_val(qval,[0,1,6,5,4])

[[ 3.108389   -4.475667    0.48932284 -1.5597734  -4.2876277   1.7763205
   1.4811139 ]
 [-1.4003955   6.1832314  -4.985691   -4.1888833  -1.5123048  -4.463805
  -3.974847  ]
 [ 2.2894852  -3.138714   -2.8857412  -1.4809004   1.8512341   4.2536697
   4.0193796 ]
 [-1.3206322  -1.2862328  -0.91365874  3.5920303  -3.6180768   0.32002065
  -2.9198515 ]
 [-1.5550622  -2.1367185   4.909155   -0.28157374  3.177616   -5.0457687
  -2.743158  ]]


/var/folders/0g/5jl4b1d12lvc6g70prn2_rnr0000gn/T/ipykernel_6158/3364995072.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor_35 = torch.tensor(tensor_array)


tensor([3.1084, 6.1832, 4.0194, 0.3200, 3.1776], requires_grad=True)

In [231]:
def linear_schedule(start_e: float, end_e: float, duration: int, t: int):
    slope = (end_e - start_e) / duration
    return max(slope * t + start_e, end_e)

In [245]:
# Reset the environment 
def training(env,terminationco2,current_state):
    for global_step in range(220):
        epsilon = linear_schedule(1,0.05,0.2*150,global_step)
        # prob is more than epsilon -> best action
        q_values = None
        if epsilon < random.random():
            q_values = q_network(torch.Tensor(current_state))
            actions = concise(q_values)

        else:
            actions = rand_action()
        
        next_state, rewards, termination, infos = env.step(actions)
# Plotting, maybe????
        # training 
        # next state
        with torch.no_grad():
            target_values = target_network(torch.Tensor(next_state))
            target_actions = torch.tensor(concise_target(target_values))
            
            # concise function
            # gamma -> 0.9
            td_target = rewards + 0.9 * target_actions * (1 - float(termination))
            td_target = torch.tensor(td_target, requires_grad=True)
        
        # old_val : actions [0,1,6,5,3]
        old_val = torch.tensor(i_to_val(q_network(torch.Tensor(current_state)),actions),requires_grad=True)
#         old_val = q_network(current_state)
        
        loss = F.mse_loss(td_target, old_val)


        # optimize the model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # target_network_frequency - 30
        # tau - 0.9
        print(global_step)
        
        if global_step % 30 == 0:
                for target_network_param, q_network_param in zip(target_network.parameters(), q_network.parameters()):
                    target_network_param.data.copy_(
                        0.9 * q_network_param.data + (0.1) * target_network_param.data
                    )
                
        current_state = next_state
        
    # Reset is maybe not needed

In [246]:
training(env1,100,state)

[[ 4.069163   -4.2297473   2.1158845  -1.6855636  -0.5255966   4.312361
   3.9806395 ]
 [-0.60727036  7.8368626  -6.4480925  -3.5299368  -0.8560422  -3.2705858
  -2.3743873 ]
 [ 1.8282933  -2.1178846  -2.1545992  -1.5179553   1.632037    5.830057
   5.271708  ]
 [ 0.06300171 -0.6672755  -1.5391257   4.413642   -1.6813574   0.615024
  -3.8939972 ]
 [-1.9126883  -2.384901    4.7259455   1.0648589   3.8635714  -3.9369802
  -2.3254523 ]]
0
[[  59.8781    -87.47422   -12.245893  -34.723923   19.811283  109.129364
   125.29541 ]
 [ -52.148155  149.50456  -151.15001  -166.44547   -19.202742  -59.649082
  -142.6853  ]
 [  46.206234  -77.91155   -90.18237    26.376678   34.897125  194.90007
   168.69289 ]
 [  26.827913 -110.363556  -27.22839    75.68873    57.136513   61.629696
  -103.76926 ]
 [ -29.118355  -72.4386    135.44695    18.796852  120.08843  -193.13852
   -14.243898]]
1
[[  60.282627  -87.944954  -12.071944  -35.011166   20.168682  109.861046
   125.806175]
 [ -52.17645   150.39555 

/var/folders/0g/5jl4b1d12lvc6g70prn2_rnr0000gn/T/ipykernel_6158/230597155.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor_35 = torch.tensor(states)
/var/folders/0g/5jl4b1d12lvc6g70prn2_rnr0000gn/T/ipykernel_6158/2219161995.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  td_target = torch.tensor(td_target, requires_grad=True)
/var/folders/0g/5jl4b1d12lvc6g70prn2_rnr0000gn/T/ipykernel_6158/3364995072.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor_35 = torch.tensor(tensor_array)
/var/folders/0g/5jl4b1d12lvc6g70prn